# Use `k3d` rendering to visualize isocontours

There are lots of rendering tool kits available for Jupyter and Python.  Making isocontours with`k3d` 
particularly easy.  To install, you only need to do:

```
$ pip install k3d
```

## Using EXP coefficients, the EXP config file to construct a basis, and slices from the FieldGenerator

Read in config from EXP yaml file and render a movie from the coefficient file.   This notebook will create the movie for either the disk or the halo by changing the component variable from 'dark' to 'star' and vice versa.

## Begin with the usual imports

You may need to append the pyEXP location to your Python path, depending on your installation.

In [1]:
import os
import copy
import yaml
# You man need something like this if pyEXP is not installed in a PYTHONPATH location
# sys.path.append('/my/path/to/site-packages')
import pyEXP
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors
from os.path import exists

import k3d
from k3d import matplotlib_color_maps

plt.rcParams['figure.figsize'] = [12, 9]

## Switch to the working directory
I like to be explicit about my working directory but you don't need to do this here.  It would be sufficient to simply pass the full path to the coefficient factory below or launch the notebook from a working directory.

In [2]:
os.chdir('/data/Nbody/CubeTest')

## Configure the basis and component

At minimum, you need to
- Define the EXP config file to import the config.   
- Define the target component.

The rest of the notebook should then run without changes.  

Two optional parameters:
- The half size of each axis
- Number of pixels along each axis

In [89]:
# key parameters
exp_config = 'config16b.yml'
component  = 'cube'  # You can make the halo movie by changing this to 'dark halo'

# options
rmin       = 0.0
rmax       = 1.0
npix       = 50

## Read the EXP config file, generate the basis from the config, and get the run tag for convenience

In [90]:
# Open and read the YAML file
#
with open(exp_config, 'r') as f:
    yaml_db = yaml.load(f, Loader=yaml.FullLoader)

In order to make sure that your YAML configuration file matches the cache file from a previously
run simulation or computed coefficient set, the `BasisFactory` requires that you specify the cache
file name.  EXP will create that name on the fly.  So, we need to add the cache file name to the
YAML configuration by hand.  This may seem a bit awkward, but this level of intentional specification
will help prevent confusion later.

Specifically in the lines below, we iterate through the `Components` stanza and add the name cache
to the configuration.  Here, that named cache is `.eof.cache.run0`.

In [91]:
# Grab both star and dark, although I'm mostly interested in star at this point
#
for v in yaml_db['Components']:
    if v['name'] == component:
        # v['force']['parameters']['eof_file'] = '.eof.cache.run001'
        # v['force']['parameters']['ncylodd'] = 3
        config = yaml.dump(v['force'])
        
print(config)
 

id: cube
parameters:
  nmaxx: 16
  nmaxy: 16
  nmaxz: 16



In [92]:
# Construct the basis instance
#
basis = pyEXP.basis.Basis.factory(config)

# Get the runtag
#
runtag = yaml_db['Global']['runtag']
print("\nRuntag from {} is: {}".format(exp_config, runtag))
coeffile = 'outcoef.{}.{}'.format(component, runtag)
print("\nCoef file is:", coeffile)


Runtag from config16b.yml is: run16b

Coef file is: outcoef.cube.run16b


## Read the coefficients

In [98]:
coefs = pyEXP.coefs.Coefs.factory(coeffile, stride=10)
print(coefs.Times())

[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68, 0.7, 0.72, 0.74, 0.76, 0.78, 0.8, 0.82, 0.84, 0.86, 0.88, 0.9, 0.92, 0.94, 0.96, 0.98, 1.0, 1.02, 1.04, 1.06, 1.08, 1.1, 1.12, 1.14, 1.16, 1.18, 1.2, 1.22, 1.24, 1.26, 1.28, 1.3, 1.32, 1.34, 1.36, 1.38, 1.4, 1.42, 1.44, 1.46, 1.48, 1.5, 1.52, 1.54, 1.56, 1.58, 1.6, 1.62, 1.64, 1.66, 1.68, 1.7, 1.72, 1.74, 1.76, 1.78, 1.8, 1.82, 1.84, 1.86, 1.88, 1.9, 1.92, 1.94, 1.96, 1.98, 2.0, 2.02, 2.04, 2.06, 2.08, 2.1, 2.12, 2.14, 2.16, 2.18, 2.2, 2.22, 2.24, 2.26, 2.28, 2.3, 2.32, 2.34, 2.36, 2.38, 2.4, 2.42, 2.44, 2.46, 2.48, 2.5, 2.52, 2.54, 2.56, 2.58, 2.6, 2.62, 2.64, 2.66, 2.68, 2.7, 2.72, 2.74, 2.76, 2.78, 2.8, 2.82, 2.84, 2.86, 2.88, 2.9, 2.92, 2.94, 2.96, 2.98, 3.0, 3.02, 3.04, 3.06, 3.08, 3.1, 3.12, 3.14, 3.16, 3.18, 3.2, 3.22, 3.24, 3.26, 3.28, 3.3, 3.32, 3.34, 3.36, 3.38, 3.4, 3.42, 3.

## Set the output field grid and render the slices

In [99]:
import time
start = time.time()

times = coefs.Times() # Limit the range to something interesting...
pmin  = [rmin, rmin, rmin]
pmax  = [rmax, rmax, rmax]
grid  = [npix, npix, npix]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

print('Created fields instance')

volumes = fields.volumes(basis, coefs)
print("Elapsed time:", time.time() - start)

Created fields instance
Elapsed time: 31979.56501531601


## This prints out the data ranges for the last frame

In [95]:
n = -1
for v in volumes[times[n]]:
    d = volumes[times[n]][v]
    print('{:20s}: min={:10.2e}  max={:10.2e}  shape={}'.format(v, np.min(d), np.max(d), d.shape))

dens                : min= -1.02e+00  max=  9.75e+01  shape=(50, 50, 50)
dens m=0            : min=  0.00e+00  max=  0.00e+00  shape=(50, 50, 50)
dens m>0            : min= -1.02e+00  max=  9.75e+01  shape=(50, 50, 50)
potl                : min= -3.89e+00  max=  6.05e-01  shape=(50, 50, 50)
potl m=0            : min=  0.00e+00  max=  0.00e+00  shape=(50, 50, 50)
potl m>0            : min= -3.89e+00  max=  6.05e-01  shape=(50, 50, 50)
x force             : min= -1.86e+01  max=  1.85e+01  shape=(50, 50, 50)
y force             : min= -1.63e+01  max=  1.62e+01  shape=(50, 50, 50)
z force             : min= -1.45e+01  max=  1.44e+01  shape=(50, 50, 50)


Now get the data from the last frame for rendering

In [96]:
dens = volumes[times[n]]['dens']
potl = volumes[times[n]]['potl']
phif = volumes[times[n]]['x force']
print("Time={} N={}".format(times[n], len(times)))

# fields.file_volumes(basis, coefs, 'fid_test')

Time=8.0 N=5


In [24]:
def repack(vol):
    """Rewrite x-y-z volume into z-y-x volume"""
    shp = vol.shape
    ret = np.ndarray((shp[2], shp[1], shp[0]))
    for i in range(shp[2]):
        for j in range(shp[1]):
            for k in range(shp[0]):
                    ret[i, j, k] = vol[k, j, i]
    return ret

And do the rendering with `k3d`:

In [97]:
plot1 = k3d.plot()

color_map = k3d.basic_color_maps.Jet

levs = [0.75, 2.0, 5.0, 10.0, 20.0, 30.0, 40.0]

minlev = 0.0
maxlev = 50.0

for lev in levs:
    plot1 += k3d.marching_cubes(repack(dens), level=lev, attribute=[lev], color_map=color_map,
                                      color_range=[minlev, maxlev],
                                      opacity=0.25,
                                      xmin=rmin, xmax=rmax,
                                      ymin=rmin, ymax=rmax,
                                      zmin=rmin, zmax=rmax,
                                      compression_level=9,
                                      flat_shading=False)
    
plot1 += k3d.text2d('T={:2.1f}'.format(times[n]), position=(0.05, 0.05))

plot1.display()

Output()

In [14]:
plot1.camera = [-0.35, -0.8, 1.5, 0.5, 0.5, 0.5, 0, 0, 1]

In [100]:
plot1.camera_auto_fit = False
plot1.grid_auto_fit = False
    
# Decorator/generator
@plot1.yield_screenshots
def makeFrames():
    global plot1 # Need this to be mutable
    icnt = 0     # Frame counter
    for t in times:
        # Remove all drawables
        count = len(plot1.objects)
        for j in range(count):
            plot1 -= plot1.objects[-1]
            
        # Add new drawables
        dens = volumes[t]['dens']
        for lev in levs:
            plot1 += k3d.marching_cubes(repack(dens), level=lev, attribute=[lev], color_map=color_map,
                                      color_range=[minlev, maxlev],
                                      opacity=0.25,
                                      xmin=rmin, xmax=rmax,
                                      ymin=rmin, ymax=rmax,
                                      zmin=rmin, zmax=rmax,
                                      compression_level=9,
                                      flat_shading=False)
        # Add time stamp
        plot1 += k3d.text2d('T={}'.format(t), position=(0.05, 0.05))
        
        # Render and print to disk
        plot1.fetch_screenshot()
        screenshot = yield
        with open('screenshot_%03d.png'%icnt, 'wb') as f:
            f.write(screenshot)
        icnt += 1
        
makeFrames()